# Introduction/Project Overview:
In this notebook, I will present my solution and analysis of the Titanic dataset. This is a very famous dataset that can be found on [kaggle](https://www.kaggle.com/c/titanic/data). The dataset contains demographics of the Titanc passengers, incluiding who survived and who did not. The goal is to build a model that can correctly classify new examples (check who will survive or not). Throughout this notebook I will visualize the data, explain some data preprocessing techniques, construct and evaluate models and analyze the results. 

#### Data Exploration & Preprocessing:  
I will go over the dataset, analyzing its various features, checking for missing values, and gaining insights into the distribution of variables. Prior to building the models, I will preprocess the data by handling missing values, encoding categorical variables, and scaling numerical features to ensure good model performance.

#### Model Building & Evaluation:
In this notebook I will try implement several models to try and correctly classify passengers who survied. This is a supervised learning task as we are given the labels of who survived and who did not. For this project the models I have chosen are logistic regression, decision trees, random forests, support vector machines and neural networks. For each of these modules I will evalute their peformance using f1score, confusion matrices, and overall accuracy. 

#### Conclusion: 
Finally, I will interpret the results of the models, identifying significant factors that contribute to passenger survival prediction and discussing potential areas for model improvement. The Titanic dataset is a good challange to test your knowledege on machine learning. This will serve as a good test for me to keep learning and testing my skills.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

# Data Exploration & Preprocessing:
As mentioned earlier I got the dataset from kaggle. The link to that can be found above. The download came with two csv files. One for the training set and one for the test set. Since I have it locally on my computer I can eassily access the data as shown below. Some of the first steps we will do before creating a model is to see what our data looks like.

In [2]:
# read train and test sets
train = pd.read_csv('./train.csv') 
test = pd.read_csv('./test.csv') 

We loaded in the data into pandas dataframes and now we want to see what our data looks like. What does vairables does it contain and what data types, etc. 

In [3]:
train.info() # get info on our train 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


The block above gives us a lot of information. For starters it tells us we have a total of 891 samples. This is a good ammount as it is a big enohgh ammount for the model to learn from but not too large where we would require lots of computing power and time for training. We also see that we have 12 columns.`Age` has 177 missing values which is a big ammount. `Cabin` has a lot of missing values too. Lastly it seems we have 5 categorical columns and 7 numerical columns. 

Lets count our missing values and then look more into our data and see what will be useful for training. 

In [4]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

As we saw `Age` has 177 missing values. `Cabin` has 2 however it might not be very useful when making a prediction on who survives the titanic but we will try to fill it. Lets breakdown each column, what they mean and if we will keep them for constructing our model. 

`PassengderId`: This column is not useful because its just a id assigned by the dataset. We will drop this column before training. 

`Survived`: This column is our labels and is important since this is a supervised machine learning problem. If we wanted to go with a clustering/unsupervised learning we could drop this. For the purpose of the notebook we will be keeping this. 

`Pclass`: On kaggle it says that this columns serves "A proxy for socio-economic status". Where 1 is upper, 2 is middle and 3 is lower. This will be important for our model

`Name`: This is simply the name of the passenger. This could be important as some family last names could mean that they are from a wealthier family and therfore might have a higher chance of surviving. We can also use this to approximate age. For this notebook we will most likely drop it. 

`Sex/Age`: The sex of the passenger. Since women and children were  prioritized in the case of an emergency this would helpful to determined who would survive. 

`SibSp`: In the kaggle description of the dataset it says that sibsp is "# of siblings / spouses aboard the Titanic". This could be helpful. 

`Parch`: In the kaggle description of the dataset it says that parch is "# of parents / children aboard the Titanic". This will also be helpful. 

`Ticket`: Simply the ticket ID so we can remove this.

`Fare`: How much they paid for their ticket. This can be useful as maybe workers did not pay for ticket and upper class people payed for their tickets. We will keep this column. 

`Cabin`: This is the cabin they were staying at. This could be useful but there are several missing values in this column so for now we will ignore it. 

`Embarked`: The Location of where they embarked. This could be important to determine who survived. For examples people who embarked at a certain location might be workers and others might be upper class familiies. This could be useful. 

Knowing how many missing values we have will be important because we will want to know if its worth filling or dropping. For example `Cabin` we will most likely drop because there is so many missing values. `Age` seems like a very important feature and not many are missing. Lets move on and get some information on the numerical columns that we have. 

In [5]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


The block above gives us a lot of information. For example we can see that the proportion of people that survived was `38%`. The mean age was `29`, the oldest person was 80 years old. Laslty average fare was `32.20`.

Now lets compare `Age`,`SibSp`,`Parch`, and `Fare` by Survival to see any relationships.

In [6]:
pd.pivot_table(train, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])

,Age,Fare,Parch,SibSp
Survived,,,,
0,30.626179,22.117887,0.329690,0.553734
1,28.343690,48.395408,0.464912,0.473684


We can see that the average `Age` of people that did not survive was `30.6` while the average `Age` of those who did was `28.34`. Although not a huge difference it makes sense that people who survived are younger as they prioritized women and children in case of emergencies. Additionally a intersting relationship here is `Fare`. For those who did not survive their average `Fare` was `22.11` while those who did survive was `48.39`. This could mean that those who payed larger ammounts of money for their fare came from wealthy families and had priority. This tells us that `Fare` could be an important feature. 

Moving on lets do the same our categorical columns. Lets start by counting the number of survivors in `Pclass` which is socioeconomic class. We add `values=PassengerId` so that we count each passanger if we do not add that we count all the survivors in `pclass` within each column. 

In [7]:
pd.pivot_table(train, index = 'Survived', columns = 'Pclass', values='PassengerId', aggfunc ='count')

Pclass,1,2,3
Survived,,,
0,80,97,372
1,136,87,119


From the table above we see that most who died are in the lower class which is `Pclass` 3 with 372 people dead. Most survivors is `Pclass` 1 which is to be expected as that is the upper class. In the middle class we have a almost even number of survivors and dead people. This will for sure be helpful in determining who sill survive. 

In [8]:
pd.pivot_table(train, index = 'Survived', columns = 'Sex', values='PassengerId', aggfunc ='count')

Sex,female,male
Survived,,
0,81,468
1,233,109


In this table above we see that most survivors were female. Most cassualties were male. This is pretty straight forward because women and children were prioritized in a emergency. 

In [9]:
pd.pivot_table(train, index = 'Survived', columns = 'Embarked', values='PassengerId', aggfunc ='count')

Embarked,C,Q,S
Survived,,,
0,75,47,427
1,93,30,217


Lastly we have embarked whcih for the most part depending on location is relatively balanced. However in location `S` most people did not survive. This could be because maybe most lower class or workers embarked at this location. Either way it will be helpful in determining who will not survive.

Now that we have explored some of our data, lets fill in some missing values and work on some feature engineering. First we are going to combine the train and test. That way all the processing can be done at once. 

In [10]:
train['train_test'] = 1 # distinguish between sets 
test['train_test'] = 0
test['Survived'] = np.NaN # fill survived with nan for test set
data = pd.concat([train,test]) # join 

We previously saw that there are two missing values for `Embarked` in the training set. There. might be more in test set but either way lets fill them. 

In [11]:
# check embarked missing values
data[data['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train_test
61,62,1.0,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,1
829,830,1.0,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,1


There is still just two. These two people survived, are part of the upper class and are female. Lets get all the women who are upper class, fare of 80 or greater and survived. Then we can count where they embarked. 

In [12]:
upper_class_women_survived = data[(data['Sex'] == 'female') & (data['Pclass'] == 1) & (data['Fare'] >= 80) & (data['Survived'] == 1)]

In [13]:
upper_class_women_survived['Embarked'].value_counts()

Embarked
C    24
S    21
Q     1
Name: count, dtype: int64

The table abve says that women who were upper class, fare greater than 80 and survived embarked at `C` or `S`. Most survivors came from `S` with a total of 217. `C` has second most at `93`. With this information we will fill in the missing values witj `S`. This was a lot for 2 missing values but is good pracice. 

In [14]:
data['Embarked'] = data['Embarked'].fillna('C')

Lets move on to filling missing `Age` values. As we saw earlier there is a good amount of missing in the training set. To fill in the missing `Age` values we are going to be using the `Name` column. Most names contain a `Mr`, `Mrs`or some other thing in their name. These people would be older because they are married or have some sort of status. This could help us see if they have better chances of surviving. What we are going to do is get the average of age of each person with some sort of title in their name. The use those values to fill in the missing ones. For example we will calculate the average age of `Mrs` then we will use that average to fill in the age for any other `Mrs`.

If they don't have that in their name and their `Parch` is zero then their age will be 0. This is because they are not a parent or child and in the kaggle datset info we are told that "Some children travelled only with a nanny, therefore parch=0 for them."

Lastly if they don't have `Mr` or `Mrs` in their name then we will get the avarage age of people who don't and use that for their missing `Age` value. First get each persons title and turn it encode it.

In [15]:
data['Name'] = data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

Below we are encoding the columns `Pclass`, `Sex`, `Embarked` and `Name`. Name however has just the titles of people

In [16]:
data = pd.get_dummies(data, columns=['Pclass', 'Sex', 'Embarked', 'Name'], prefix=['Pclass', 'Sex', 'Embarked', 'Name'], dtype=float) # encode the columns

In [17]:
data.info() # check our new columns

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PassengerId        1309 non-null   int64  
 1   Survived           891 non-null    float64
 2   Age                1046 non-null   float64
 3   SibSp              1309 non-null   int64  
 4   Parch              1309 non-null   int64  
 5   Ticket             1309 non-null   object 
 6   Fare               1308 non-null   float64
 7   Cabin              295 non-null    object 
 8   train_test         1309 non-null   int64  
 9   Pclass_1           1309 non-null   float64
 10  Pclass_2           1309 non-null   float64
 11  Pclass_3           1309 non-null   float64
 12  Sex_female         1309 non-null   float64
 13  Sex_male           1309 non-null   float64
 14  Embarked_C         1309 non-null   float64
 15  Embarked_Q         1309 non-null   float64
 16  Embarked_S         1309 non-nu

Now all that's left is to fill in the missing values for `Age`. 

In [18]:
name_columns = ['Name_Capt', 'Name_Col', 'Name_Don', 'Name_Dona', 'Name_Dr', 'Name_Jonkheer', 
                'Name_Lady', 'Name_Major', 'Name_Master', 'Name_Miss', 'Name_Mlle', 'Name_Mme',
                'Name_Mr', 'Name_Mrs', 'Name_Ms', 'Name_Rev', 'Name_Sir', 'Name_the Countess']

In [19]:
average_ages = {} # collect averages ages in a dictionary

In [21]:
for name_column in name_columns:
    filtered_df = data[data[name_column] == 1]    
    avg_age = filtered_df['Age'].mean()
    average_ages[name_column] = avg_age

In [22]:
average_ages

{'Name_Capt': 70.0,
 'Name_Col': 54.0,
 'Name_Don': 40.0,
 'Name_Dona': 39.0,
 'Name_Dr': 43.57142857142857,
 'Name_Jonkheer': 38.0,
 'Name_Lady': 48.0,
 'Name_Major': 48.5,
 'Name_Master': 5.482641509433963,
 'Name_Miss': 21.774238095238097,
 'Name_Mlle': 24.0,
 'Name_Mme': 24.0,
 'Name_Mr': 32.25215146299484,
 'Name_Mrs': 36.99411764705882,
 'Name_Ms': 28.0,
 'Name_Rev': 41.25,
 'Name_Sir': 49.0,
 'Name_the Countess': 33.0}

In [23]:
def fill_age(df, mean_ages):
    for index, row in df.iterrows():
        if np.isnan(row['Age']):
            name = row['Name'].split(',')[1].split('.')[0].strip()
            print(name)
            if name in mean_ages:
                df.loc[index, 'Age'] = mean_ages[name]
            elif row['Parch'] == 0:
                df.loc[index, 'Age'] = 0
    return df

In [24]:
fill_age(data, average_ages) # fill the age column if nan

KeyError: 'Name'

In [ ]:
train = train.fillna(train.loc[:, 'Age'].mean()) # fill missing values with average age

Now lets change the name column. What we are going to do is strip the name and only keep the persons title. So stuff like `Mrs` will be the name if the have it. Once we have that we will encode the column.  

We now have all the data that we need. We also got rid of the `Name` column because we don't really need it anymore. Below we are going to check and see if our dataset balanced. Because we are trying to classify `Survived` we need to make sure we have a good ammount of examples of each class `1` or `0`. If not our model will only learn from one of the classes more than the other. 

In [ ]:
train['Survived'].value_counts() # seems slightly unbalanced but should be fine

Another step before we get to modeling is we need to encode the `Pclass`, `Sex`, and `Embarked`.

Now we should normalize the `Age` and `Fare` columns. 

In [ ]:
from sklearn.preprocessing import StandardScaler

We now have all the data as we want it. We encoded the `Pclass`, `Sex` and `Embarked` columns. We filled missing `Age` values. We have two dataframes to test if filling missing values was worth it. Now we need do apply the same transformations that we did to the training set to the test set. Then we can start modeling.

In [ ]:
passenger_id = test.pop('PassengerId') 

In [ ]:
test.isnull().sum()

In [ ]:
fill_age(test, mean_age_mrs, mean_age_mr) # fill the age column using average values from ealier

In [ ]:
test = test.fillna(train.loc[:, 'Fare'].mean()) # since its just one missing value we will fill with mean of train

In [ ]:
test.drop(['Ticket', 'Cabin'], axis=1, inplace=True) # drop columns in test dataframe

In [ ]:
test['Name'] = test['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [ ]:
X_test = pd.get_dummies(test, columns=['Pclass', 'Sex', 'Embarked', 'Name'], prefix=['Pclass', 'Sex', 'Embarked', 'Name'], dtype=float) # encode the columns

In [ ]:
X_test['Age'] = scaler.fit_transform(X_test[['Age']])
X_test['Fare'] = scaler.fit_transform(X_test[['Fare']])

In [ ]:
X_test.info()

At this point I realised that we dont have labels for the test set. So I am just going to model and evaluate on the training dataset and leave the predictions for the test. 

# Model Building

As mentioned in the project description, I was going to use logistic regression, random forests and neural networks for this dataset. But since there labels dont come with the test dataset I am just going to use random forests and a neural network since I think it will be the best peforming. I will then print the training accuracy and submit the predictions into kaggle. After that I will use whichever has a highest training accuracy. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
y_train = train.pop('Survived') 
X_train = train 

In [ ]:
rf = RandomForestClassifier(random_state=42) # a classifer to use grid search on

In [ ]:
param_grid = { # the parameters to test
    'n_estimators': [100, 200],  
    'max_depth': [10, 20, None],  
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4],  
    'bootstrap': [True, False]  
} 

In [ ]:
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_ # print best parameters

In [ ]:
clf = RandomForestClassifier(n_estimators=100,
                             max_depth=None,
                             bootstrap=True,
                             min_samples_leaf=1,
                             min_samples_split=10,
                             random_state=42) # create the classifier

In [ ]:
clf.fit(X_train, y_train) # fitting the data

In [ ]:
y_pred = clf.predict(X_train) # getting predictions on training data

In [ ]:
accuracy = accuracy_score(y_train, y_pred) # getting accuracy on training data

In [ ]:
conf_matrix = confusion_matrix(y_train, y_pred)
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)

The accuracy score is 0.91 which is not bad. Its not terribly low but not very high either so its not underfitting or overfitting. Now we can get the predictions and then just submit them. 

In [ ]:
predictions = clf.predict(X_test) # predictions on the test set

In [ ]:
output = pd.DataFrame({'PassengerId': passenger_id.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)

# Conclusion
So ideally I would have checked the data in the `test.csv` to see that the labels are missing. Using the training set to evaluate is not really a good measurement. However seeing that it did not get a .99 on the accuracy might be a good thing because we know its not overfitting. To confirm this we could use the test if we had the labels. Some future improvments things for next time are to check both training and test set. So in the end this notebook serves more as a pandas excercise to get use to using all its features and was really helpful honestly. Additionally I got 0.77751 on kaggle for accuracy so not a terrible experiment.  